In [42]:
import polars as pl
import sys, os
from sippy import *
import numpy as np
from sippy import functionset as fset
from sippy import functionsetSIM as fsetSIM
import matplotlib.pyplot as plt
import pandas as pd
import control as cnt
import json
from datetime import datetime, timezone
import plotly.graph_objects as go
from plotly_resampler import FigureResampler, FigureWidgetResampler

In [43]:
data_path = '/home/alqua/data/data_vdfs'

In [44]:
start_filter_date = datetime(2024, 1, 9, 00, 00, 0, tzinfo=timezone.utc)
end_filter_date = datetime(2024, 1, 15, 00, 00, 0, tzinfo=timezone.utc)

In [45]:
p1_power = pl.read_parquet(data_path + '/' +'pump1_power_siso.par')
p3_power = pl.read_parquet(data_path + '/' +'pump3_power_siso.par')
p4_power = pl.read_parquet(data_path + '/' +'pump4_power_siso.par')
outflow_df = pl.read_parquet(data_path + '/' +'outflow_miso.par')
pressure_df = pl.read_parquet(data_path + '/' +'pressure_miso.par')


In [46]:
sampling_time = '60s'

p1_power = p1_power.filter(
                                 (pl.col('time') >= start_filter_date) &
                                   (pl.col('time')<= end_filter_date)
                                 )

p3_power = p3_power.filter(
                                 (pl.col('time') >= start_filter_date) &
                                   (pl.col('time')<= end_filter_date)
                                 )

p4_power = p4_power.filter(
                                 (pl.col('time') >= start_filter_date) &
                                   (pl.col('time')<= end_filter_date)
                                 )


pressure_df = pressure_df.filter(
                                 (pl.col('time') >= start_filter_date) &
                                   (pl.col('time')<= end_filter_date)
                                 )

In [47]:
sysid_df = outflow_df.join(p1_power, 
                left_on='time', 
                right_on='time').join(p3_power, 
                left_on='time', 
                right_on='time').join(
                p4_power, 
                left_on='time', 
                right_on='time').join(pressure_df, 
                left_on='time', 
                right_on='time').upsample(
                    time_column='time', 
                    every=sampling_time, 
                    maintain_order=True).group_by_dynamic(
                        'time', 
                        every=sampling_time, 
                        ).agg(pl.all().mean())

In [48]:
sysid_df.head(5)

time,level,outflow,pump1_speed,pump4_speed,pump3_speed,pump1_speed_right,pump1_power,pump3_speed_right,pump3_power,pump4_speed_right,pump4_power,outflow_right,pressure,level_right
"datetime[ns, UTC]",f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2024-01-09 10:31:00 UTC,164.0,595.0,1170.0,0.0,0.0,1170.0,56.700001,0.0,0.0,0.0,0.0,595.0,0.599,164.0
2024-01-09 10:32:00 UTC,164.0,595.0,1170.0,0.0,0.0,1170.0,56.599998,0.0,0.0,0.0,0.0,595.0,0.611,164.0
2024-01-09 10:33:00 UTC,165.0,611.0,1200.0,0.0,0.0,1200.0,58.299999,0.0,0.0,0.0,0.0,611.0,0.628,165.0
2024-01-09 10:34:00 UTC,165.0,625.0,1200.0,0.0,0.0,1200.0,58.400002,0.0,0.0,0.0,0.0,625.0,0.596,165.0
2024-01-09 10:35:00 UTC,166.0,628.0,1230.0,0.0,0.0,1230.0,61.799999,0.0,0.0,0.0,0.0,628.0,0.651,166.0


In [49]:
features_cols = {
    'time': 'time_utc',  
    'level': 'h',
    'outflow': 'qout',
    'pump1_speed': 'p1_rpm',
    'pump4_speed': 'p4_rpm',
    'pump3_speed': 'p3_rpm',
    'pump1_power': 'p1_power',
    'pump3_power': 'p3_power',
    'pump4_power': 'p4_power',
    'pressure': 'pressure',
}

sysid_df = (
    sysid_df
    .select(features_cols.keys())
    .rename(features_cols)
    .filter(
        (pl.col("p1_rpm") > 0) |
        (pl.col("p3_rpm") > 0) |
        (pl.col("p4_rpm") > 0)
    )
)




# Exclude rows where all p1_rpm, p3_rpm, p4_rpm are greater than zero
sysid_df = sysid_df.with_columns((
    pl.when(pl.col("p1_rpm") > 0).then(pl.col("qout")).otherwise(0).alias("qout_p1"), 
    pl.when(pl.col("p3_rpm") > 0).then(pl.col("qout")).otherwise(0).alias("qout_p3"),
    pl.when(pl.col("p4_rpm") > 0).then(pl.col("qout")).otherwise(0).alias("qout_p4")
)).filter(~(
    (pl.col("p1_rpm") > 0) &
    (pl.col("p3_rpm") > 0) &
    (pl.col("p4_rpm") > 0)
))

In [50]:
from sippy import functionset as fset

def identify_system(df, u_col, y_col, test_size=0.6, na=1, nb=2, theta=0, dt=None, predict_test =False, nsteps_ahead=1, plot_results=False, save_tf=False):

    selected_data = df.select([u_col, y_col])
    split_point = int(len(selected_data) * test_size)
    train_df = selected_data.head(split_point)
    test_df = selected_data.tail(len(selected_data) - split_point) 
    


    u_train, u_test = train_df[u_col].to_numpy(), test_df[u_col].to_numpy()
    y_train, y_test = train_df[y_col].to_numpy(), test_df[y_col].to_numpy()
    
    na_ords = [na]         
    nb_ords = [[nb]]       
    theta = [[theta]] 

    id_ARX = system_identification(y_train, u_train, 'ARX', stab_cons=True, 
                                ARX_orders=[na_ords, nb_ords, theta], tsample=dt) 
    
    G = id_ARX.G  
    print(f"\nTransfer function from {u_col} to {y_col}:")
    print("==================")
    print(id_ARX.G)
    if save_tf: 
        tf_data = {
                    "u": u_col,
                    "y": y_col,
                    "na": na, 
                    "nb":nb,
                    "num": [round(x, 5) for x in id_ARX.NUMERATOR[0][0]],
                    "den": [round(x, 5) for x in id_ARX.DENOMINATOR[0][0][1:]],
                    "dt": dt
                }
        
        filename = f"tf_{u_col}_to_{y_col}.json"
        result_path = "results/"
        with open(result_path + filename, 'w') as f:
            json.dump(tf_data, f, indent=4)

    if predict_test: 
        t_test = np.arange(0, len(y_test)) * dt
        Yval = fset.validation(id_ARX, u_test, y_test, t_test, k=nsteps_ahead)
        if plot_results: 
            fig = FigureWidgetResampler(go.Figure())
            fig.update_layout(margin=dict(l=10, r=10, t=10, b=10))
            fig.add_trace(
                        go.Scattergl(
                            x=t_test,
                            y=y_test,
                            name=f'{y_col} (Predicted from {u_col})',  # Shows column names
                            showlegend=True,
                            mode='lines'
                        )
                    )
            fig.add_trace(
                        go.Scattergl(
                            x=t_test,
                            y=Yval.flatten(),
                            name=f'{y_col} (Predicted from {u_col})',  # Shows column names
                            showlegend=True,
                            mode='lines'
                        )
                    )
            fig.update_layout(height=200, template="plotly_dark")
            display(fig)
            
        
        return id_ARX, G, t_test, Yval

    
    else: 
        return id_ARX, G


In [51]:
columns = [
    ("p1_rpm", "qout_p1"),
    ("p1_rpm", "p1_power"),

    ("p3_rpm", "qout_p3"),
    ("p3_rpm", "p3_power"),

    ("p4_rpm", "qout_p4"),
    ("p4_rpm", "p4_power"),
]


for u_col, y_col in columns:
    print(f"Identifying model {u_col}, {y_col}")
    id_ARX, G, t_test, Yval = identify_system(
        df=sysid_df, 
        u_col=u_col,
        y_col=y_col,
        test_size=0.4, 
        na=1,
        nb=1,
        theta=0,
        dt=1,
        predict_test=True, 
        nsteps_ahead=1, 
        plot_results=True, 
        save_tf=True
    )
    # Process/save result as needed

Identifying model p1_rpm, qout_p1

Transfer function from p1_rpm to qout_p1:

  0.1892
----------
z - 0.6125

dt = 1



FigureWidgetResampler({
    'data': [{'mode': 'lines',
              'name': ('<b style="color:sandybrown">[R' ... 'i style="color:#fc9944">~5</i>'),
              'showlegend': True,
              'type': 'scattergl',
              'uid': '2e66df9f-5f97-4149-9518-6df7445cc4cd',
              'x': array([   0,    2,    7, ..., 4780, 4789, 4790]),
              'y': array([539., 530., 609., ..., 906., 831., 861.])},
             {'mode': 'lines',
              'name': ('<b style="color:sandybrown">[R' ... 'i style="color:#fc9944">~5</i>'),
              'showlegend': True,
              'type': 'scattergl',
              'uid': '0096bc21-0257-4c0b-a3c5-b295e6249223',
              'x': array([   0,    1,    8, ..., 4782, 4789, 4790]),
              'y': array([  0.        , 540.14907563, 594.37743487, ..., 755.59828998,
                          744.79760237, 710.49570827])}],
    'layout': {'height': 200, 'margin': {'b': 10, 'l': 10, 'r': 10, 't': 10}, 'template': '...'}
})

Identifying model p1_rpm, p1_power

Transfer function from p1_rpm to p1_power:

 0.01719
----------
z - 0.6242

dt = 1



FigureWidgetResampler({
    'data': [{'mode': 'lines',
              'name': ('<b style="color:sandybrown">[R' ... 'i style="color:#fc9944">~5</i>'),
              'showlegend': True,
              'type': 'scattergl',
              'uid': '5057b217-46c6-4e9e-8d90-145dae88e534',
              'x': array([   0,    1,    7, ..., 4782, 4787, 4790]),
              'y': array([49.29999924, 51.5       , 57.90000153, ..., 49.79999924, 48.70000076,
                          51.09999847])},
             {'mode': 'lines',
              'name': ('<b style="color:sandybrown">[R' ... 'i style="color:#fc9944">~5</i>'),
              'showlegend': True,
              'type': 'scattergl',
              'uid': '179cc07b-607f-4285-905c-cdfec98e5cd4',
              'x': array([   0,    1,    8, ..., 4781, 4786, 4790]),
              'y': array([ 0.        , 49.8555324 , 56.25492711, ..., 52.22735636, 48.76528284,
                          49.26909615])}],
    'layout': {'height': 200, 'margin': {'b': 10,

Identifying model p3_rpm, qout_p3

Transfer function from p3_rpm to qout_p3:

  0.2901
----------
z - 0.3748

dt = 1



...

FigureWidgetResampler({
    'data': [{'mode': 'lines',
              'name': ('<b style="color:sandybrown">[R' ... 'i style="color:#fc9944">~5</i>'),
              'showlegend': True,
              'type': 'scattergl',
              'uid': 'be02f393-55c5-49bd-92cc-fb85eb37f5a4',
              'x': array([   0,    1,    5, ..., 4780, 4785, 4790]),
              'y': array([0., 0., 0., ..., 0., 0., 0.])},
             {'mode': 'lines',
              'name': ('<b style="color:sandybrown">[R' ... 'i style="color:#fc9944">~5</i>'),
              'showlegend': True,
              'type': 'scattergl',
              'uid': 'f6af64f9-fd97-45a4-8a63-348cb5963117',
              'x': array([   0,    1,    5, ..., 4780, 4785, 4790]),
              'y': array([0., 0., 0., ..., 0., 0., 0.])}],
    'layout': {'height': 200, 'margin': {'b': 10, 'l': 10, 'r': 10, 't': 10}, 'template': '...'}
})

Identifying model p4_rpm, qout_p4

Transfer function from p4_rpm to qout_p4:

  0.309
----------
z - 0.3992

dt = 1



FigureWidgetResampler({
    'data': [{'mode': 'lines',
              'name': ('<b style="color:sandybrown">[R' ... 'i style="color:#fc9944">~5</i>'),
              'showlegend': True,
              'type': 'scattergl',
              'uid': 'ffff6dfb-783b-4d5b-90f9-f76a32806cc1',
              'x': array([   0,    1,    5, ..., 4780, 4789, 4790]),
              'y': array([  0.,   0.,   0., ..., 906., 831., 861.])},
             {'mode': 'lines',
              'name': ('<b style="color:sandybrown">[R' ... 'i style="color:#fc9944">~5</i>'),
              'showlegend': True,
              'type': 'scattergl',
              'uid': '18010d3b-e97b-4a33-b195-79fb3e4ad66d',
              'x': array([   0,    1,    5, ..., 4782, 4789, 4790]),
              'y': array([  0.        ,   0.        ,   0.        , ..., 666.34716098,
                          656.83268249, 634.47940115])}],
    'layout': {'height': 200, 'margin': {'b': 10, 'l': 10, 'r': 10, 't': 10}, 'template': '...'}
})

Identifying model p4_rpm, p4_power

Transfer function from p4_rpm to p4_power:

 0.01379
----------
z - 0.7015

dt = 1



FigureWidgetResampler({
    'data': [{'mode': 'lines',
              'name': ('<b style="color:sandybrown">[R' ... 'i style="color:#fc9944">~5</i>'),
              'showlegend': True,
              'type': 'scattergl',
              'uid': 'ab7e9d15-e4d9-4432-aa1d-1ce2db8269b8',
              'x': array([   0,    1,    5, ..., 4781, 4785, 4790]),
              'y': array([ 0.        ,  0.        ,  0.        , ..., 43.20000076, 43.70000076,
                          42.5       ])},
             {'mode': 'lines',
              'name': ('<b style="color:sandybrown">[R' ... 'i style="color:#fc9944">~5</i>'),
              'showlegend': True,
              'type': 'scattergl',
              'uid': '27e7ba6d-a8c6-407b-8332-68124bd38c62',
              'x': array([   0,    1,    5, ..., 4782, 4786, 4790]),
              'y': array([ 0.        ,  0.        ,  0.        , ..., 44.01770936, 44.27191867,
                          42.98274787])}],
    'layout': {'height': 200, 'margin': {'b': 10,